# Module Calculations - Teste e Validação

Notebook para testar e validar cálculos de módulos fotovoltaicos.
Baseado no código do pvlib-service.

In [1]:
# Instalação das dependências necessárias
!pip install pandas numpy requests matplotlib seaborn jupyter ipython
!pip install pvlib-python scipy scikit-learn

# Verificar instalação
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import seaborn as sns

print("✅ Todas as dependências foram instaladas com sucesso!")
print(f"📊 Pandas versão: {pd.__version__}")
print(f"🔢 NumPy versão: {np.__version__}")
print(f"🌐 Requests disponível")
print(f"📈 Matplotlib disponível")
print(f"🎨 Seaborn disponível")

ERROR: Could not find a version that satisfies the requirement pvlib-python (from versions: none)
ERROR: No matching distribution found for pvlib-python
✅ Todas as dependências foram instaladas com sucesso!
📊 Pandas versão: 2.3.2
🔢 NumPy versão: 2.3.2
🌐 Requests disponível
📈 Matplotlib disponível
🎨 Seaborn disponível


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import pvlib
from pvlib import pvsystem, modelchain, location, inverter
from pvlib.temperature import TEMPERATURE_MODEL_PARAMETERS
import warnings
warnings.filterwarnings('ignore')

# Configurações de plotagem
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['font.size'] = 10

print(f"📚 PVLIB Version: {pvlib.__version__}")

ImportError: cannot import name 'pv' from 'pvlib' (/Users/rubensfilho/Desktop/Workspace/Soft House/Besspro/python/venv/lib/python3.11/site-packages/pvlib/__init__.py)

## 📍 Configuração da Localização

Configure aqui as coordenadas para análise dos módulos fotovoltaicos.

In [ ]:
# =============================================================================
# 🌍 CONFIGURE SUAS COORDENADAS AQUI:
# =============================================================================

# Coordenadas padrão (São Paulo - SP)
LATITUDE = -23.5505
LONGITUDE = -46.6333

# Exemplos de outras coordenadas (descomente para usar):
# Rio de Janeiro - RJ
# LATITUDE, LONGITUDE = -22.9068, -43.1729

# Brasília - DF
# LATITUDE, LONGITUDE = -15.7975, -47.8919

# Belo Horizonte - MG
# LATITUDE, LONGITUDE = -19.9191, -43.9378

# Fortaleza - CE
# LATITUDE, LONGITUDE = -3.7319, -38.5267

# Porto Alegre - RS
# LATITUDE, LONGITUDE = -30.0346, -51.2177

# Suas coordenadas personalizadas:
# LATITUDE, LONGITUDE = SEU_LAT, SEU_LON

print(f"🎯 ANÁLISE DE MÓDULOS FOTOVOLTAICOS")
print(f"="*50)
print(f"📍 Coordenadas selecionadas: {LATITUDE:.4f}, {LONGITUDE:.4f}")
print(f"🌍 Sistema irá buscar dados PVGIS diretamente da API")

## 📍 Dados de Entrada para Cálculo de Módulos
Defina aqui os parâmetros do sistema:

In [ ]:
# PARÂMETROS DO SISTEMA - EDITE AQUI
CONSUMO_ANUAL_KWH = 6000  # Consumo anual em kWh
TILT = 23  # Inclinação dos painéis
AZIMUTH = 0  # Azimute (0 = Norte)

# Módulo fotovoltaico
MODULO = {
    'fabricante': 'Canadian Solar',
    'modelo': 'CS3W-540MS',
    'potencia_nominal_w': 540,
    'vmp': 41.4,
    'imp': 13.04,
    'voc': 49.9,
    'isc': 13.87,
    'coef_temp_pmax': -0.35,
    'coef_temp_voc': -0.27,
    'coef_temp_isc': 0.05,
    'noct': 42
}

# Inversor
INVERSOR = {
    'fabricante': 'SMA',
    'modelo': 'STP15000TL-30',
    'potencia_saida_ca_w': 15000,
    'potencia_fv_max_w': 22500,
    'tensao_cc_max_v': 1000,
    'numero_mppt': 2,
    'strings_por_mppt': 3,
    'vdco': 620,
    'pso': 65,
    'c0': -0.000008,
    'c1': -0.000002,
    'c2': 0.001,
    'c3': -0.013,
    'pnt': 0.02
}

# Perdas do sistema
PERDAS = {
    'temperatura': 8,
    'sombreamento': 3,
    'mismatch': 2,
    'cabeamento': 2,
    'sujeira': 5,
    'inversor': 3,
    'outras': 2
}

print("🔧 Configuração do Sistema:")
print(f"⚡ Consumo anual: {CONSUMO_ANUAL_KWH:,} kWh")
print(f"📐 Orientação: Tilt={TILT}°, Azimute={AZIMUTH}°")
print(f"🔋 Módulo: {MODULO['fabricante']} {MODULO['modelo']} ({MODULO['potencia_nominal_w']}W)")
print(f"🔌 Inversor: {INVERSOR['fabricante']} {INVERSOR['modelo']} ({INVERSOR['potencia_saida_ca_w']/1000}kW)")
print(f"📉 Perdas totais: {sum(PERDAS.values())}%")

## 🛠️ Funções de Cálculo de Módulos (extraídas do pvlib-service)

In [ ]:
class ModuleCalculationService:
    """Serviço para cálculos de módulos - Baseado no pvlib-service"""
    
    def __init__(self):
        pass
    
    def calculate_optimal_modules(self, lat: float, lon: float, consumo_anual: float,
                                 modulo: dict, inversor: dict, perdas: dict,
                                 tilt: float = 0, azimuth: float = 0,
                                 num_modules: int = None) -> dict:
        """Calcular sistema fotovoltaico otimizado"""
        
        print(f"🚀 Iniciando cálculo de sistema fotovoltaico...")
        print(f"📍 Local: {lat}, {lon}")
        print(f"⚡ Consumo: {consumo_anual:,} kWh/ano")
        
        # Criar localização
        loc = location.Location(latitude=lat, longitude=lon, tz='UTC')
        
        # Buscar e preparar dados meteorológicos do PVGIS
        weather_data, monthly_data = self._prepare_weather_data(lat, lon, tilt, azimuth)
        
        # Determinar número de módulos
        if num_modules is None:
            num_modules = self._calculate_module_count(consumo_anual, modulo, monthly_data)
        
        print(f"🔋 Número de módulos: {num_modules}")
        
        # Criar sistema PV
        system = self._create_pv_system(modulo, inversor, num_modules, tilt, azimuth)
        
        # Executar ModelChain
        mc = modelchain.ModelChain(system, loc, aoi_model='physical', 
                                  spectral_model='no_loss')
        
        print("⚙️ Executando simulação PVLIB...")
        mc.run_model(weather_data)
        
        # Aplicar perdas do sistema
        energia_bruta = mc.results.ac.sum() / 1000  # kWh
        fator_perdas = 1 - sum(perdas.values()) / 100
        energia_liquida = energia_bruta * fator_perdas
        
        # Calcular métricas
        potencia_total = (num_modules * modulo['potencia_nominal_w']) / 1000  # kW
        cobertura = (energia_liquida / consumo_anual) * 100
        yield_especifico = energia_liquida / potencia_total  # kWh/kWp
        fator_capacidade = energia_liquida / (potencia_total * 8760) * 100
        
        # Análise mensal
        energia_mensal = mc.results.ac.resample('M').sum() / 1000 * fator_perdas
        energia_por_ano = mc.results.ac.resample('Y').sum() / 1000 * fator_perdas
        
        resultado = {
            'num_modulos': num_modules,
            'potencia_total_kw': potencia_total,
            'energia_total_anual_kwh': round(energia_liquida, 1),
            'energia_por_modulo': round(energia_liquida / num_modules, 1),
            'cobertura_percentual': round(cobertura, 1),
            'yield_especifico': round(yield_especifico, 1),
            'fator_capacidade': round(fator_capacidade, 2),
            'energia_mensal': energia_mensal.tolist(),
            'energia_por_ano': energia_por_ano.tolist(),
            'perdas_aplicadas': perdas,
            'fator_perdas': fator_perdas,
            'energia_bruta_kwh': round(energia_bruta, 1),
            'irradiacao_media': round(monthly_data['media_anual'], 2),
            'parametros_sistema': {
                'modulo': modulo,
                'inversor': inversor,
                'orientacao': {'tilt': tilt, 'azimuth': azimuth}
            },
            'dados_pvgis': monthly_data
        }
        
        print("✅ Cálculo concluído!")
        print(f"🔸 Potência instalada: {potencia_total:.2f} kWp")
        print(f"🔸 Geração anual: {energia_liquida:.1f} kWh")
        print(f"🔸 Cobertura: {cobertura:.1f}%")
        print(f"🔸 Yield específico: {yield_especifico:.1f} kWh/kWp")
        
        return resultado
    
    def _prepare_weather_data(self, lat: float, lon: float, tilt: float = 0, azimuth: float = 0) -> tuple:
        """Buscar dados PVGIS usando a mesma API do notebook 01"""
        
        print("🌐 Buscando dados PVGIS (seriescalc) diretamente da API...")
        
        # Usar a mesma API do notebook 01_pvgis_data_analysis
        url = "https://re.jrc.ec.europa.eu/api/v5_2/seriescalc"
        params = {
            'lat': lat,
            'lon': lon,
            'startyear': 2005,
            'endyear': 2020,
            'outputformat': 'json',
            'usehorizon': 1,
            'selectrad': 1,
            'angle': tilt,
            'aspect': azimuth
        }
        
        try:
            print(f"🔄 Fazendo requisição PVGIS: {url}")
            print(f"📊 Parâmetros: {params}")
            
            response = requests.get(url, params=params, timeout=120)
            response.raise_for_status()
            data = response.json()
            
            # Verificar estrutura dos dados
            if 'outputs' not in data:
                raise ValueError("Estrutura de dados PVGIS inesperada - sem 'outputs'")
            
            # Extrair dados horários
            hourly_data = data['outputs'].get('hourly', [])
            if not hourly_data:
                print("⚠️ Nenhum dado horário encontrado, tentando TMY...")
                return self._fallback_tmy_data(lat, lon)
            
            print(f"✅ Dados PVGIS recebidos: {len(hourly_data)} registros")
            
            # Processar dados como no notebook 01
            df = pd.DataFrame(hourly_data)
            df['timestamp'] = pd.to_datetime(df['time'], format='%Y%m%d:%H%M', errors='coerce')
            
            # Converter colunas numéricas
            for col in ['G(i)', 'H_sun', 'T2m', 'WS10m', 'Int']:
                if col in df.columns:
                    df[col] = pd.to_numeric(df[col], errors='coerce')
            
            # Filtrar dados válidos
            df = df.dropna(subset=['timestamp', 'G(i)'])
            df = df.set_index('timestamp')
            
            # Calcular médias mensais (mesmo método do notebook 01)
            monthly = df.groupby(df.index.month).agg({
                'G(i)': 'mean',
                'H_sun': 'mean', 
                'T2m': 'mean',
                'WS10m': 'mean'
            }).round(2)
            
            monthly_data = {
                'irradiacao_mensal': monthly['G(i)'].tolist(),
                'temperatura_media': monthly['T2m'].tolist(),
                'vento_medio': monthly['WS10m'].tolist(),
                'horas_sol': monthly['H_sun'].tolist(),
                'media_anual': monthly['G(i)'].mean(),
                'maximo': monthly['G(i)'].max(),
                'minimo': monthly['G(i)'].min(),
                'variacao_sazonal': monthly['G(i)'].max() - monthly['G(i)'].min()
            }
            
            # Preparar dados para PVLIB
            weather = pd.DataFrame({
                'ghi': df['G(i)'],  # PVGIS fornece irradiância no plano inclinado
                'dni': df['G(i)'] * 0.8,  # Estimativa DNI
                'dhi': df['G(i)'] * 0.2,  # Estimativa DHI  
                'temp_air': df['T2m'],
                'wind_speed': df['WS10m']
            })
            
            # Usar apenas dados de um ano para simulação
            weather = weather[weather.index.year == weather.index.year.min()]
            
            print(f"📊 Dados processados: {len(weather)} registros")
            print(f"☀️ Irradiação média: {monthly_data['media_anual']:.2f} kWh/m²/dia")
            print(f"🌡️ Temperatura média: {np.mean(monthly_data['temperatura_media']):.1f}°C")
            
            return weather, monthly_data
            
        except Exception as e:
            print(f"❌ Erro ao buscar dados PVGIS seriescalc: {e}")
            print("🔄 Tentando API TMY como fallback...")
            return self._fallback_tmy_data(lat, lon)
    
    def _fallback_tmy_data(self, lat: float, lon: float) -> tuple:
        """Fallback usando TMY se seriescalc falhar"""
        
        try:
            url = "https://re.jrc.ec.europa.eu/api/v5_2/tmy"
            params = {'lat': lat, 'lon': lon, 'outputformat': 'json'}
            
            response = requests.get(url, params=params, timeout=60)
            response.raise_for_status()
            data = response.json()
            
            if 'outputs' in data and 'tmy_hourly' in data['outputs']:
                hourly_data = data['outputs']['tmy_hourly']
                print(f"✅ Dados TMY recebidos: {len(hourly_data)} registros")
                
                df = pd.DataFrame(hourly_data)
                df['timestamp'] = pd.to_datetime(df['time(UTC)'], format='%Y%m%d:%H%M')
                df = df.set_index('timestamp')
                
                for col in ['G(h)', 'T2m', 'WS10m', 'RH']:
                    if col in df.columns:
                        df[col] = pd.to_numeric(df[col], errors='coerce')
                
                # Calcular médias mensais
                monthly = df.groupby(df.index.month).agg({
                    'G(h)': 'mean',
                    'T2m': 'mean',
                    'WS10m': 'mean'
                }).round(2)
                
                monthly_data = {
                    'irradiacao_mensal': monthly['G(h)'].tolist(),
                    'temperatura_media': monthly['T2m'].tolist(), 
                    'vento_medio': monthly['WS10m'].tolist(),
                    'horas_sol': [8] * 12,  # Estimativa
                    'media_anual': monthly['G(h)'].mean(),
                    'maximo': monthly['G(h)'].max(),
                    'minimo': monthly['G(h)'].min(),
                    'variacao_sazonal': monthly['G(h)'].max() - monthly['G(h)'].min()
                }
                
                weather = pd.DataFrame({
                    'ghi': df.get('G(h)', 500),
                    'dni': df.get('G(h)', 500) * 0.8,
                    'dhi': df.get('G(h)', 500) * 0.2,
                    'temp_air': df.get('T2m', 25),
                    'wind_speed': df.get('WS10m', 2)
                })
                
                return weather.dropna(), monthly_data
            
        except Exception as e:
            print(f"❌ Erro no fallback TMY: {e}")
        
        # Último fallback: dados sintéticos
        print("📝 Usando dados meteorológicos sintéticos...")
        dates = pd.date_range('2020-01-01', '2020-12-31', freq='h')
        
        # Dados baseados na latitude (mais realista)
        base_ghi = 600 if abs(lat) < 23.5 else 500
        seasonal_var = 200 * np.cos(2 * np.pi * np.arange(len(dates)) / (24 * 365.25))
        daily_var = 300 * np.maximum(0, np.sin(np.pi * (np.arange(len(dates)) % 24) / 24))
        
        weather = pd.DataFrame({
            'ghi': base_ghi + seasonal_var + daily_var,
            'dni': 600,
            'dhi': 100,
            'temp_air': 25 + 5 * np.sin(2 * np.pi * np.arange(len(dates)) / (24 * 365.25)),
            'wind_speed': 2.5
        }, index=dates)
        
        # Monthly data sintético
        monthly_data = {
            'irradiacao_mensal': [base_ghi/1000 * 24/1000] * 12,  # kWh/m²/dia
            'temperatura_media': [25] * 12,
            'vento_medio': [2.5] * 12,
            'horas_sol': [8] * 12,
            'media_anual': base_ghi/1000 * 24/1000,
            'maximo': base_ghi/1000 * 24/1000 * 1.2,
            'minimo': base_ghi/1000 * 24/1000 * 0.8,
            'variacao_sazonal': base_ghi/1000 * 24/1000 * 0.4
        }
        
        return weather, monthly_data
    
    def _calculate_module_count(self, consumo_anual: float, modulo: dict, monthly_data: dict) -> int:
        """Calcular número de módulos necessário baseado nos dados PVGIS"""
        
        # Usar dados processados do PVGIS
        irradiacao_media = monthly_data['media_anual']  # kWh/m²/dia
        hsp_anual = irradiacao_media * 365  # kWh/m²/ano
        
        # Fator de perdas
        fator_perdas = 1 - sum(PERDAS.values()) / 100
        
        # Energia por kWp instalado 
        energia_por_kwp = hsp_anual * fator_perdas
        
        # Potência necessária
        potencia_necessaria_kw = consumo_anual / energia_por_kwp
        
        # Número de módulos
        num_modules = int(np.ceil((potencia_necessaria_kw * 1000) / modulo['potencia_nominal_w']))
        
        print(f"📊 Estimativa baseada em dados PVGIS:")
        print(f"🔸 Irradiação média: {irradiacao_media:.2f} kWh/m²/dia")
        print(f"🔸 HSP anual: {hsp_anual:.0f} kWh/m²/ano")
        print(f"🔸 Energia por kWp: {energia_por_kwp:.0f} kWh/kWp/ano")
        print(f"🔸 Potência necessária: {potencia_necessaria_kw:.2f} kWp")
        
        return num_modules
    
    def _create_pv_system(self, modulo: dict, inversor: dict, num_modules: int,
                         tilt: float, azimuth: float) -> pvsystem.PVSystem:
        """Criar sistema PV"""
        
        # Parâmetros do módulo
        module_params = {
            'pdc0': modulo['potencia_nominal_w'],
            'v_mp': modulo['vmp'],
            'i_mp': modulo['imp'],
            'v_oc': modulo['voc'],
            'i_sc': modulo['isc'],
            'alpha_sc': modulo['coef_temp_isc'] / 100,
            'beta_voc': modulo['coef_temp_voc'] / 100,
            'gamma_pdc': modulo['coef_temp_pmax'] / 100,
            'cells_in_series': 120,
            'temp_ref': 25
        }
        
        # Parâmetros do inversor
        inverter_params = {
            'pdc0': inversor['potencia_fv_max_w'],
            'eta_inv_nom': 0.96,
            'eta_inv_ref': 0.9637
        }
        
        # Criar arrays
        arrays = [
            pvsystem.Array(
                mount=pvsystem.FixedMount(surface_tilt=tilt, surface_azimuth=azimuth),
                module_parameters=module_params,
                modules_per_string=1,
                strings=num_modules
            )
        ]
        
        return pvsystem.PVSystem(arrays=arrays, inverter_parameters=inverter_params)
    
    def analyze_performance(self, resultado: dict) -> dict:
        """Análise detalhada de performance"""
        
        energia_mensal = np.array(resultado['energia_mensal'])
        
        analise = {
            'energia_diaria_media': energia_mensal.mean() / 30,
            'energia_diaria_max': energia_mensal.max() / 30,
            'energia_diaria_min': energia_mensal.min() / 30,
            'variabilidade_mensal': energia_mensal.std() / energia_mensal.mean() * 100,
            'meses_acima_media': sum(energia_mensal > energia_mensal.mean()),
            'eficiencia_sistema': resultado['fator_capacidade'],
            'performance_ratio': resultado['yield_especifico'] / resultado['irradiacao_media'] / 365 * 100 if 'irradiacao_media' in resultado else 80
        }
        
        return analise

## ⚡ Executar Cálculos de Módulos

In [ ]:
# Inicializar serviço
module_service = ModuleCalculationService()

# Executar cálculo (agora busca dados PVGIS diretamente)
resultado = module_service.calculate_optimal_modules(
    lat=LATITUDE,
    lon=LONGITUDE,
    consumo_anual=CONSUMO_ANUAL_KWH,
    modulo=MODULO,
    inversor=INVERSOR,
    perdas=PERDAS,
    tilt=TILT,
    azimuth=AZIMUTH
)

print("\n" + "="*50)
print("📊 RESULTADOS DO SISTEMA FOTOVOLTAICO")
print("="*50)
print(f"🔋 Número de módulos: {resultado['num_modulos']}")
print(f"⚡ Potência instalada: {resultado['potencia_total_kw']:.2f} kWp")
print(f"🌟 Geração anual líquida: {resultado['energia_total_anual_kwh']:,.1f} kWh")
print(f"📈 Cobertura do consumo: {resultado['cobertura_percentual']:.1f}%")
print(f"🎯 Yield específico: {resultado['yield_especifico']:.1f} kWh/kWp")
print(f"🔄 Fator de capacidade: {resultado['fator_capacidade']:.2f}%")
print(f"💡 Energia por módulo: {resultado['energia_por_modulo']:.1f} kWh/ano")
print(f"☀️ Irradiação média: {resultado['irradiacao_media']:.2f} kWh/m²/dia")
print(f"📉 Perdas totais: {(1-resultado['fator_perdas'])*100:.1f}%")

# Dados detalhados mensais e totais anuais
print("\n" + "="*50)
print("📅 GERAÇÃO MENSAL E TOTAIS DETALHADOS")
print("="*50)

meses = ['Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho', 
         'Julho', 'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro']

# Calcular totais
energia_mensal = resultado['energia_mensal']
total_anual = sum(energia_mensal)
media_mensal = total_anual / 12

print(f"📊 RESUMO ANUAL:")
print(f"   🎯 Total anual: {total_anual:,.1f} kWh")
print(f"   📊 Média mensal: {media_mensal:,.1f} kWh/mês")
print(f"   📈 Maior mês: {max(energia_mensal):,.1f} kWh ({meses[energia_mensal.index(max(energia_mensal))]})")
print(f"   📉 Menor mês: {min(energia_mensal):,.1f} kWh ({meses[energia_mensal.index(min(energia_mensal))]})")
print(f"   🔄 Variação: {(max(energia_mensal) - min(energia_mensal)):,.1f} kWh")

print(f"\n📅 GERAÇÃO MENSAL DETALHADA:")
for i, (mes, energia) in enumerate(zip(meses, energia_mensal)):
    # Percentual em relação à média
    percentual = (energia / media_mensal - 1) * 100
    sinal = "+" if percentual > 0 else ""
    
    # Geração diária média do mês
    dias_mes = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31][i]
    energia_diaria = energia / dias_mes
    
    print(f"   {mes:>10}: {energia:>7.1f} kWh ({sinal}{percentual:>+4.1f}% vs média) | {energia_diaria:>5.1f} kWh/dia")

# Dados PVGIS relacionados (se disponível)
if 'dados_pvgis' in resultado:
    pvgis_data = resultado['dados_pvgis']
    print(f"\n☀️ DADOS PVGIS CORRESPONDENTES:")
    print(f"   📊 Irradiação média anual: {pvgis_data['media_anual']:.2f} kWh/m²/dia")
    print(f"   🌡️ Temperatura média anual: {np.mean(pvgis_data['temperatura_media']):.1f}°C")
    print(f"   💨 Vento médio anual: {np.mean(pvgis_data['vento_medio']):.1f} m/s")
    print(f"   🌞 Horas de sol médias: {np.mean(pvgis_data['horas_sol']):.1f} h/dia")
    
    print(f"\n📊 IRRADIAÇÃO MENSAL PVGIS:")
    for i, mes in enumerate(['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 
                           'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez']):
        irradiacao = pvgis_data['irradiacao_mensal'][i]
        temperatura = pvgis_data['temperatura_media'][i]
        print(f"   {mes}: {irradiacao:>5.2f} kWh/m²/dia | {temperatura:>4.1f}°C")

In [ ]:
# Análise de performance
performance = module_service.analyze_performance(resultado)

print("\n" + "="*50)
print("📈 ANÁLISE DE PERFORMANCE")
print("="*50)
print(f"☀️ Geração diária média: {performance['energia_diaria_media']:.1f} kWh/dia")
print(f"📊 Geração diária máxima: {performance['energia_diaria_max']:.1f} kWh/dia")
print(f"📊 Geração diária mínima: {performance['energia_diaria_min']:.1f} kWh/dia")
print(f"🔄 Variabilidade mensal: {performance['variabilidade_mensal']:.1f}%")
print(f"📅 Meses acima da média: {performance['meses_acima_media']}/12")
print(f"⚡ Performance ratio: {performance['performance_ratio']:.1f}%")

## 📊 Visualizações dos Resultados

In [ ]:
# Gráficos dos resultados
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

meses = ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez']
energia_mensal = resultado['energia_mensal']
consumo_mensal = [CONSUMO_ANUAL_KWH / 12] * 12

# 1. Geração vs Consumo mensal
x = np.arange(len(meses))
width = 0.35

ax1.bar(x - width/2, energia_mensal, width, label='Geração', color='gold', alpha=0.8)
ax1.bar(x + width/2, consumo_mensal, width, label='Consumo', color='steelblue', alpha=0.8)
ax1.set_title('Geração vs Consumo Mensal', fontsize=14, fontweight='bold')
ax1.set_ylabel('kWh')
ax1.set_xticks(x)
ax1.set_xticklabels(meses, rotation=45)
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Breakdown de perdas
perdas_labels = list(PERDAS.keys())
perdas_values = list(PERDAS.values())
colors = plt.cm.Set3(np.linspace(0, 1, len(perdas_labels)))

ax2.pie(perdas_values, labels=perdas_labels, autopct='%1.1f%%', colors=colors)
ax2.set_title('Breakdown de Perdas do Sistema', fontsize=14, fontweight='bold')

# 3. Geração acumulada anual
geracao_acumulada = np.cumsum(energia_mensal)
consumo_acumulado = np.cumsum(consumo_mensal)

ax3.plot(meses, geracao_acumulada, marker='o', linewidth=3, label='Geração Acumulada', color='green')
ax3.plot(meses, consumo_acumulado, marker='s', linewidth=3, label='Consumo Acumulado', color='red')
ax3.set_title('Geração vs Consumo Acumulado', fontsize=14, fontweight='bold')
ax3.set_ylabel('kWh')
ax3.tick_params(axis='x', rotation=45)
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Indicadores chave
indicadores = ['Yield\n(kWh/kWp)', 'Cap. Factor\n(%)', 'Cobertura\n(%)', 'PR\n(%)']
valores = [
    resultado['yield_especifico'],
    resultado['fator_capacidade'],
    resultado['cobertura_percentual'],
    performance['performance_ratio']
]

bars = ax4.bar(indicadores, valores, color=['skyblue', 'lightgreen', 'coral', 'gold'], alpha=0.8)
ax4.set_title('Indicadores de Performance', fontsize=14, fontweight='bold')
ax4.set_ylabel('Valor')

# Adicionar valores nas barras
for bar, valor in zip(bars, valores):
    height = bar.get_height()
    ax4.annotate(f'{valor:.1f}',
                xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 3),  # 3 points vertical offset
                textcoords="offset points",
                ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

# Resumo financeiro básico
print("\n💰 RESUMO ECONÔMICO BÁSICO:")
tarifa_kwh = 0.75  # R$/kWh - valor estimado
economia_anual = resultado['energia_total_anual_kwh'] * tarifa_kwh
investimento_estimado = resultado['potencia_total_kw'] * 4500  # R$/kWp estimado
payback_simples = investimento_estimado / economia_anual

print(f"💡 Economia anual estimada: R$ {economia_anual:,.2f}")
print(f"💸 Investimento estimado: R$ {investimento_estimado:,.2f}")
print(f"⏱️ Payback simples: {payback_simples:.1f} anos")
print(f"📊 ROI em 25 anos: {(economia_anual * 25 - investimento_estimado) / investimento_estimado * 100:.0f}%")

## 💾 Salvar Resultados

In [ ]:
# Salvar resultados completos
resultados_completos = {
    'sistema': resultado,
    'performance': performance,
    'configuracao': {
        'consumo_anual_kwh': CONSUMO_ANUAL_KWH,
        'modulo': MODULO,
        'inversor': INVERSOR,
        'perdas': PERDAS,
        'orientacao': {'tilt': TILT, 'azimuth': AZIMUTH}
    },
    'localização': {'latitude': LATITUDE, 'longitude': LONGITUDE}
}

# Salvar em JSON
try:
    import os
    os.makedirs('../data', exist_ok=True)
    
    with open('../data/module_calculation_results.json', 'w') as f:
        json.dump(resultados_completos, f, indent=2, default=str)
    
    print("💾 Resultados salvos em:")
    print("- ../data/module_calculation_results.json")
    print("\n✅ Dados de módulos prontos para análise financeira!")
    
except Exception as e:
    print(f"⚠️ Erro ao salvar resultados: {e}")
    print("📄 Resultados calculados com sucesso, mas não foi possível salvar em arquivo")